# MLOps with CLI V2

- Creating Scripts for MLOps Pipeline
- The kernel you use for this notebook can be your job_env or the Python 3.10 SDK V2

## In this notebook we will:

- Create base folders for holding the scripts in your pipeline steps
- Create a pipeline to create and register a model
- Register the raw data (which your pipeline will transform)
- Create scripts for pipeline steps to prep your data, train your model, evaluate your model
- Create scripts for deploying your model with your devOps pipeline

You can use the **Python 3.10 - SDK V2** environment to run this notebook, as it is just creating the files that are needed for your Azure DevOps pipeline

To work with this notebook you will need to have the following:
- An Azure DevOps Service Connection set to: `aml-dev`
- An Azure DevOps Service Connection set to: `aml-qa`
- An Azure DevOps Environment set to qa
- An Azure DevOps Variable Group Called: `devops-variable-group-dev`
    - Variables in the Azure DevOps Variable Group include:
        - `location` (ex: eastus)
        - `resourceGroup` (the name of your AMLS workspace resource group connected with your aml-dev service connection)
        - `wsName` (the name of your AMLS workspace resource group connected with your aml-dev service connection)
- An Azure DevOps Variable Group Called: `devops-variable-group-qa`
    - Variables in the Azure DevOps Variable Group include:
        - `location` (ex: eastus)
        - `resourceGroup` (the name of your AMLS workspace resource group connected with your aml-dev service connection)
        - `wsName` (the name of your AMLS workspace resource group connected with your aml-dev service connection)
        
Locations are defined here: https://github.com/microsoft/azure-pipelines-extensions/blob/master/docs/authoring/endpoints/workspace-locations

In [1]:
#import required libraries
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.entities import Environment, BuildContext

In [2]:
subscription_id = '<SUBSCRIPTION_ID>'
resource_group= '<resource_group>'
workspace = '<workspace>'

subscription_id =  "5da07161-3770-4a4b-aa43-418cbbb627cf"
resource_group =  "mm-ch9aml-dev-rg"
workspace =  "mm-ch9aml-dev"

## Connecting to your workspace

In [3]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [4]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

In [5]:
df= pd.read_csv('./data/titanic.csv')
print(df.shape)
print(df.columns)

try:
    registered_data_asset = ml_client.data.get(name='titanic_raw', version=1)
    print('data asset is registered')
except:
    print('register data asset')
    my_data = Data(
        path="./data/titanic.csv",
        type=AssetTypes.URI_FILE,
        description="Titanic CSV",
        name="titanic_raw",
        version="1",
    )

    ml_client.data.create_or_update(my_data)

(891, 12)
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
data asset is registered


In [6]:
import os

# Create a folder for the experiment files
script_folder = './src/'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'src folder created')


# Create a folder for the experiment files
script_folder = './src/prep'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'prep folder created')

script_folder = './src/train'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'train folder created')

script_folder = './src/eval'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'eval folder created')

script_folder = './src/deploy'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'deploy folder created')

script_folder = './src/pipeline'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'pipeline folder created')

script_folder = './src/conda-yamls'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'conda-yamls folder created')


./src/ src folder created
./src/prep prep folder created
./src/train train folder created
./src/eval eval folder created
./src/deploy deploy folder created
./src/pipeline pipeline folder created
./src/conda-yamls conda-yamls folder created


## Creating Compute Cluster for Training in Dev Environment

In [7]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "cpu-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4, idle_time_before_scale_down = 3600
    )
    ml_client.compute.begin_create_or_update(compute)

## Preparing Scripts for Training Pipelines

In [8]:
%%writefile ./src/conda-yamls/pipeline_conda_env.yml
name: job_env
dependencies:
- python=3.10
- scikit-learn=1.1.3
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults==1.48.0 #needed for the inferece schema
  - mlflow<=1.30.0
  - azure-ai-ml==1.1.2
  - mltable==1.0.0
  - azureml-mlflow==1.48.0

Overwriting ./src/conda-yamls/pipeline_conda_env.yml


## Prep Data.py

In [9]:
%%writefile ./src/prep/prep.py

import argparse
import pandas as pd

parser = argparse.ArgumentParser("prep")
parser.add_argument("--raw_data", type=str, help="Path to raw data")
parser.add_argument("--prep_data", type=str, help="Path of prepped data")
args = parser.parse_args()

print(args.raw_data)
print(args.prep_data)

df = pd.read_csv(args.raw_data + '/titanic.csv')

df['Age'] = df.groupby(['Pclass', 'Sex'])['Age'].apply(lambda x: x.fillna(x.median()))
df['Sex']= df['Sex'].apply(lambda x: x[0] if pd.notnull(x) else 'X')
df['Loc']= df['Cabin'].apply(lambda x: x[0] if pd.notnull(x) else 'X')
df.drop(['Cabin', 'Ticket'], axis=1, inplace=True)
df['Embarked'] = df['Embarked'].fillna('S')
df.loc[:,'GroupSize'] = 1 + df['SibSp'] + df['Parch']

df_train = df
df = df_train.drop(['Name','SibSp', 'Parch', 'PassengerId'], axis=1)

df.to_csv(args.prep_data)

Overwriting ./src/prep/prep.py


## Create train.py file

In [10]:
%%writefile ./src/train/train.py
import os
import argparse
import joblib
import shutil
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature
from mlflow.utils.environment import _mlflow_conda_env
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score


# define functions
def main(args):
    current_run = mlflow.start_run()
    mlflow.sklearn.autolog(log_models=False)

    # read in data
    print('about to read file:' + args.prep_data)
    df = pd.read_csv(args.prep_data)
    model, X_test, y_test = model_train('Survived', df, 0)
    
    model_file = os.path.join(args.model_output, 'titanic_model.pkl')
    joblib.dump(value=model, filename=model_file)
    
    os.makedirs("outputs", exist_ok=True)
    y_test.to_csv('outputs/Y_test.csv', index = False)
    X_test.to_csv( 'outputs/X_test.csv', index = False)
    shutil.copytree('./outputs/', args.test_data, dirs_exist_ok=True)

def model_train(LABEL, df, randomstate):
    print('df.columns = ')
    print(df.columns)
    
    df['Embarked'] = df['Embarked'].astype(object)
    df['Loc'] = df['Loc'].astype(object)
    df['Loc'] = df['Sex'].astype(object)
    df['Pclass'] = df['Pclass'].astype(float)
    df['Age'] = df['Age'].astype(float)
    df['Fare'] = df['Fare'].astype(float)
    df['GroupSize'] = df['GroupSize'].astype(float)
    
    y_raw           = df[LABEL]
    columns_to_keep = ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
    X_raw           = df[columns_to_keep]
    

    print(X_raw.columns)
     # Train test split
    X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, test_size=0.2, random_state=args.randomstate)
    
    #use Logistic Regression estimator from scikit learn
    lg = LogisticRegression(penalty='l2', C=1.0, solver='liblinear')
    preprocessor = buildpreprocessorpipeline(X_train)
    
    #estimator instance
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', lg)], verbose=True)

    model = clf.fit(X_train, y_train)
    
    print('type of X_test = ' + str(type(X_test)))
          
    y_pred = model.predict(X_test)
    
    print('*****X_test************')
    print(X_test)
    
    #get the active run.
    run = mlflow.active_run()
    print("Active run_id: {}".format(run.info.run_id))

    acc = model.score(X_test, y_test )
    print('Accuracy:', acc)
    MlflowClient().log_metric(run.info.run_id, "test_acc", acc)
    
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test,y_scores[:,1])
    print('AUC: ' , auc)
    MlflowClient().log_metric(run.info.run_id, "test_auc", auc)
    
    
    # Signature
    signature = infer_signature(X_test, y_test)

    # Conda environment
    custom_env =_mlflow_conda_env(
        additional_conda_deps=["scikit-learn==1.1.3"],
        additional_pip_deps=["mlflow<=1.30.0"],
        additional_conda_channels=None,
    )

    # Sample
    input_example = X_train.sample(n=1)

    # Log the model manually
    mlflow.sklearn.log_model(model, 
                             artifact_path="championmodel", 
                             conda_env=custom_env,
                             signature=signature,
                             input_example=input_example)

    return model, X_test, y_test


def buildpreprocessorpipeline(X_raw):

    categorical_features = X_raw.select_dtypes(include=['object', 'bool']).columns
    numeric_features = X_raw.select_dtypes(include=['float','int64']).columns

    categorical_transformer = Pipeline(steps=[('onehotencoder', 
                                               OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore'))])


    numeric_transformer1 = Pipeline(steps=[('scaler1', SimpleImputer(missing_values=np.nan, strategy = 'mean'))])
    

    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric1', numeric_transformer1, numeric_features),
            ('categorical', categorical_transformer, categorical_features)], remainder='drop')
    
    return preprocessor



def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--prep_data", default="data", type=str, help="Path to prepped data, default to local folder")
    parser.add_argument("--input_file_name", type=str, default="titanic.csv")
    parser.add_argument("---randomstate", type=int, default=42)
    
    parser.add_argument("--model_output", type=str, help="Path of output model")
    parser.add_argument("--test_data", type=str,)

    # parse args
    args = parser.parse_args()
    print(args.prep_data)
    print(args.input_file_name)
    print(args.randomstate)
    print(args.model_output)
    print(args.test_data)
    # return args
    return args


# run script
if __name__ == "__main__":
    # parse args
    args = parse_args()

    # run main function
    main(args)

Overwriting ./src/train/train.py


## Evaluate Model

In [11]:
%%writefile ./src/eval/evaluatemodel.py
import argparse
import os
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature
from mlflow.utils.environment import _mlflow_conda_env
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score
import joblib
import shutil
from azureml.core import Run
from azureml.core import Model

    
    
# define functions
def main(args):
    
    model_name = args.model_name
    
    run = Run.get_context()
    ws = run.experiment.workspace
    run_id = run.id
    print('run_id =' + run_id)
    model_list = Model.list(ws, name=model_name, latest=True)
    first_registration = len(model_list)==0
    current_model = None
        
    # read in data
    print('about to read file:' + args.test_data)
    X_test = pd.read_csv(args.test_data + '/X_test.csv')
    df_y_test = pd.read_csv(args.test_data + '/Y_test.csv')
    y_test  = df_y_test.values.flatten()
    #load champion model
    model_file = os.path.join(args.model_folder, 'titanic_model.pkl')
    champion_model = joblib.load(model_file)
    
    y_pred_current = champion_model.predict(X_test)
    print('y_pred_current')
    print(y_pred_current)
    print('y_test')
    print(y_test)
    
    champion_auc = roc_auc_score(y_test,y_pred_current)
    print('champion_auc:' , champion_auc)
    
    champion_acc = np.average(y_pred_current == y_test)
    print('champion_acc:', champion_acc)

    
    try:
        current_model_aml = Model(ws,args.model_name)
        os.makedirs("current_model", exist_ok=True)
        current_model_aml.download("current_model",exist_ok=True)
        current_model = mlflow.sklearn.load_model(os.path.join("current_model",args.model_name))
    except:
        print('no model register with name' + args.model_name)
        pass
    
    if current_model:
        y_pred_current = current_model.predict(X_test)
        current_acc = np.average(y_pred_current == y_test)
        print('current_acc:', current_acc)
        if champion_acc >= current_acc:
            print('better model found, registering')

            # Signature
            signature = infer_signature(X_test, y_test)

            # Conda environment
            custom_env =_mlflow_conda_env(
                additional_conda_deps=["scikit-learn==1.1.3"],
                additional_pip_deps=["mlflow<=1.30.0"],
                additional_conda_channels=None,
            )

            # Sample
            input_example = X_test.sample(n=1)

            # Log the model manually
            mlflow.sklearn.log_model(champion_model, 
                                     artifact_path=args.model_name, 
                                     conda_env=custom_env,
                                     signature=signature,
                                     input_example=input_example)
            model_uri = f'runs:/{run_id}/{args.model_name}'
            mlflow.register_model(model_uri,args.model_name)
            ##########################################################
    
            
        else:
            print('current model performs better than champion model ')
            print('champion_acc:', champion_acc)
            print('current_acc:', current_acc)
    else:
        print('no current model')
        print("First time model train, registering")

        signature = infer_signature(X_test, y_test)

        # Conda environment
        custom_env =_mlflow_conda_env(
                additional_conda_deps=["scikit-learn==1.1.3"],
                additional_pip_deps=["mlflow<=1.30.0"],
                additional_conda_channels=None,
            )

        # Sample
        input_example = X_test.sample(n=1)

        # Log the model manually
        mlflow.sklearn.log_model(champion_model, 
                                 artifact_path=args.model_name, 
                                 conda_env=custom_env,
                                 signature=signature,
                                 input_example=input_example)
        model_uri = f'runs:/{run_id}/{args.model_name}'
        mlflow.register_model(model_uri,args.model_name)
        

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--test_data", default="data", type=str, help="Path to test data")
    parser.add_argument("--model_folder", default="data", type=str, help="Path to model data")
    parser.add_argument("--model_name",default='mmchapter9titanic',type=str, help="Name of the model in workspace")

    # parse args
    args = parser.parse_args()
    
    print(args.test_data)
    print(args.model_folder)
    print(args.model_name)
        
    return args


# run script
if __name__ == "__main__":
    # parse args
    args = parse_args()

    # run main function
    main(args)


Overwriting ./src/eval/evaluatemodel.py


## AML Pipeline Definition

- This pipeline defines the AML Pipeline job to:
    - Prep
    - Train
    - Evaluate and Register

In [12]:
%%writefile ./src/pipeline/aml_train_and_eval_pipeline.yml

$schema: https://azuremlschemas.azureedge.net/latest/pipelineJob.schema.json
type: pipeline
display_name: Training_and_eval_pipeline
compute: azureml:cpu-cluster


jobs:
  prep_job:
    type: command
    code: ../prep
    command: >-
      python prep.py 
      --raw_data ${{inputs.raw_data}}
      --prep_data ${{outputs.prep_data}}
    inputs:
      raw_data:
        type: uri_folder
        path: azureml:titanic_raw:1
        mode: ro_mount
    outputs:
      prep_data:
        type: uri_file
        path: azureml://datastores/workspaceblobstore/paths/titanic_prep_data/titanic_prepped.csv
        mode: rw_mount
    environment:
      conda_file: ../conda-yamls/pipeline_conda_env.yml
      image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest

    
  train_job:
    type: command
    inputs:
      prep_data: ${{parent.jobs.prep_job.outputs.prep_data}}
    outputs:
      model_output:
        type: uri_folder
        path: azureml://datastores/workspaceblobstore/paths/titanic_model_data/
        mode: rw_mount
      test_data: 
        type: uri_folder
        path: azureml://datastores/workspaceblobstore/paths/titanic_test_data/
        mode: rw_mount
    code: ../train
    environment:
      conda_file: ../conda-yamls/pipeline_conda_env.yml
      image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest
    compute: azureml:cpu-cluster
    command: >-
      python train.py 
      --prep_data ${{inputs.prep_data}} 
      --model_output ${{outputs.model_output}}
      --test_data ${{outputs.test_data}}

  eval_job:
    type: command
    inputs:
      test_data: ${{parent.jobs.train_job.outputs.test_data}}
      model_folder: ${{parent.jobs.train_job.outputs.model_output}}
    code: ../eval
    environment:
      conda_file: ../conda-yamls/pipeline_conda_env.yml
      image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest
    compute: azureml:cpu-cluster
    command: >-
      python evaluatemodel.py 
      --test_data ${{inputs.test_data}} 
      --model_folder ${{inputs.model_folder}}

Overwriting ./src/pipeline/aml_train_and_eval_pipeline.yml


# Create Deployment files

### Be sure to change your endpoint name to be something that wil be unique

- change the vlaue of **name** to be something unique to ensure the deployment will succeed

In [13]:
%%writefile ./src/deploy/create-endpoint.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json 
name: xmmchapter9titanicendpoint
auth_mode: key 

Overwriting ./src/deploy/create-endpoint.yml


In [14]:
%%writefile ./src/deploy/create-endpoint-dev.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json 
name: xmmchapter9titanicendpointdev
auth_mode: key 

Overwriting ./src/deploy/create-endpoint-dev.yml


### Be sure to change your endpoint name to match the endpoint name in the `create-endpoint.yml` file

- change the vlaue of **endpoint_name** to be the same as in the create-endpoint.yml file

In [15]:
%%writefile ./src/deploy/model_deployment.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json 
name: green
endpoint_name: xmmchapter9titanicendpoint
model: azureml:mmchapter9titanic@latest 
instance_type: Standard_DS2_v2 
instance_count: 1  
    

Overwriting ./src/deploy/model_deployment.yml


In [16]:
%%writefile ./src/deploy/model_deployment-dev.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json 
name: green
endpoint_name: xmmchapter9titanicendpointdev
model: azureml:mmchapter9titanic@latest 
instance_type: Standard_DS2_v2 
instance_count: 1  
    

Overwriting ./src/deploy/model_deployment-dev.yml


## Create Azure DevOps Pipeline
### Be sure to change your endpoint name to match the endpoint name in the `create-endpoint.yml` file

In [17]:
%%writefile ./src/AzureDevOpsPipeline.yml

resources:
  containers:
  - container: mlops
    image: mcr.microsoft.com/mlops/python:latest

pr: none
trigger:
  branches:
    include:
    - main

variables:
- group: devops-variable-group-dev
- group: devops-variable-group-qa
- name: model_name
  value: mmchapter9titanic

- name: ENDPT_NAME
  value: xmmchapter9titanicendpoint
    
- name: DEV_ENDPT_NAME
  value: xmmchapter9titanicendpointdev


pool:
  vmImage: ubuntu-latest

stages:
- stage: 'DevRunPipline'
  variables:
  - group: devops-variable-group-dev
  displayName: 'DevTrainingPipeline'
  jobs:
  - job: "TrainingPipeline"
    steps:  
      - task: AzureCLI@1
        env:
          wsName: $(wsName)
          resourceGroup: $(resourceGroup)
          location: $(location)
        inputs:
          azureSubscription: aml-dev
          scriptLocation: inlineScript
          workingDirectory: '$(Build.SourcesDirectory)'
          inlineScript: |
            echo "files:"
            ls
            az version
            az extension add -n ml -y
            az version
            az configure --defaults group=$(resourceGroup) workspace=$(wsName) location=$(location)
            az ml model list -w $(wsName) -g $(resourceGroup)  -n $(model_name) --query "[0].version" -o tsv
            
            if [[ -z "$(az ml model list -w $(wsName) -g $(resourceGroup)  -n $(model_name) --query '[0].version' -o tsv)" ]]; then 
                echo "no model was found, set value to 0"
                echo "##vso[task.setvariable variable=modelversion;isOutput=true]0"
                echo "model does not yet exist in this environment, set the value of the model version to 0"
                exit 0
            else
                echo "model was found"
                echo "##vso[task.setvariable variable=modelversion;isOutput=true]$(az ml model list -w $(wsName) -g $(resourceGroup)  -n $(model_name) --query '[0].version' -o tsv)"
                exit 0
            fi
        name: 'setversion'
        displayName: 'Get Initial Model Version'
            
      - task: Bash@3
        inputs:
          workingDirectory: '$(Build.SourcesDirectory)'
          targetType: 'inline'
          script: |
            echo 'modelversion'
            echo $(setversion.modelversion)
            
      - task: AzureCLI@1
        timeoutInMinutes: 45
        env:
          wsName: $(wsName)
          resourceGroup: $(resourceGroup)
          location: $(location)
        inputs:
          azureSubscription: aml-dev
          scriptLocation: inlineScript
          workingDirectory: '$(Build.SourcesDirectory)'
          inlineScript: |
            echo "initial model version"
            echo $(setversion.modelversion)
            echo "files"
            ls
            az ml job create --file 'chapter 9/src/pipeline/aml_train_and_eval_pipeline.yml' --stream --set settings.force_rerun=True
        displayName: 'Training Pipeline'
            
      - task: AzureCLI@1
        env:
          wsName: $(wsName)
          subscriptionId: $(subscriptionId)
          resourceGroup: $(resourceGroup)
          location: $(location)
        inputs:
          azureSubscription: aml-dev
          scriptLocation: inlineScript
          workingDirectory: '$(Build.SourcesDirectory)'
          inlineScript: |
            echo "files:"
            ls
            az version
            az configure --defaults group=$(resourceGroup) workspace=$(wsName) location=$(location)
            az ml model list -w $(wsName) -g $(resourceGroup)  -n $(model_name) --query "[0].version" -o tsv
            echo "##vso[task.setvariable variable=finalmodelversion;isOutput=true]$(az ml model list -w $(wsName) -g $(resourceGroup)  -n $(model_name) --query '[0].version' -o tsv)"
            echo "##vso[task.setvariable variable=devResourceGroup;isOutput=true]$(resourceGroup)"
            echo "##vso[task.setvariable variable=devWsName;isOutput=true]$(wsName)"
            echo "##vso[task.setvariable variable=devLocation;isOutput=true]$(location)"
        name: 'setfinalversion'
        displayName: 'Get Final Model Version'
            
      - task: AzureCLI@1
        env:
          wsName: $(wsName)
          subscriptionId: $(subscriptionId)
          resourceGroup: $(resourceGroup)
          location: $(location)
        inputs:
          azureSubscription: aml-dev
          scriptLocation: inlineScript
          workingDirectory: '$(Build.SourcesDirectory)'
          inlineScript: |
            echo 'initial model version '$(setversion.modelversion)
            echo 'final model version   '$(setfinalversion.finalmodelversion)
            if [[ $(setversion.modelversion) == $(setfinalversion.finalmodelversion) ]]; then
                echo "##vso[task.setvariable variable=runme;isOutput=true]false"
                exit 0
            else
                echo "deploy updated model"
                echo "##vso[task.setvariable variable=runme;isOutput=true]true"
                az ml model download  -w $(wsName) -g $(resourceGroup)  -n $(model_name) -v $(setfinalversion.finalmodelversion)
            fi
        name: 'checkversions'
        displayName: 'Check Versions'
            
      - task: AzureCLI@1
        env:
          wsName: $(wsName)
          subscriptionId: $(subscriptionId)
          resourceGroup: $(resourceGroup)
          location: $(location)
        inputs:
          azureSubscription: aml-dev
          scriptLocation: inlineScript
          workingDirectory: '$(Build.SourcesDirectory)'
          inlineScript: |
            echo "check versions"
            echo $(checkversions.runme)
            
      - task: AzureCLI@1
        env:
          wsName: $(wsName)
          resourceGroup: $(resourceGroup)
          location: $(location)
        inputs:
          azureSubscription: aml-dev
          scriptLocation: inlineScript
          workingDirectory: '$(Build.SourcesDirectory)'
          inlineScript: |
            set -e
            echo 'final model version was:'$(setfinalversion.finalmodelversion)
            export NEW_DEPLOYMENT_NAME=deployment`echo $(setfinalversion.finalmodelversion)`v`echo $(date +"%s")`
 
            echo $NEW_DEPLOYMENT_NAME
            echo $ENDPT_NAME
            
            az extension add -n ml -y
            az version
            az configure --defaults group=$(resourceGroup) workspace=$(wsName) location=$(location)

            ENDPOINT_EXISTS=$(az ml online-endpoint list -g $(resourceGroup) -w $(wsName) -o tsv --query "[?name=='$DEV_ENDPT_NAME'][name]" |  wc -l)
            if [[ ENDPOINT_EXISTS -ne 1 ]]; then
                echo "endpoint does not exists"
                az ml online-endpoint create --file 'chapter 9/src/deploy/create-endpoint-dev.yml' -g $(resourceGroup) -w $(wsName) -n $(DEV_ENDPT_NAME)
                echo "creating online deployment"
                az ml online-deployment create --name $NEW_DEPLOYMENT_NAME -f 'chapter 9/src/deploy/model_deployment-dev.yml' -g $(resourceGroup) -w $(wsName) 
                echo "updating online endpoint tags"
                az ml online-endpoint update -n $DEV_ENDPT_NAME --set tags.prod=$NEW_DEPLOYMENT_NAME  --traffic "$NEW_DEPLOYMENT_NAME=100" -g $(resourceGroup) -w $(wsName)
                exit 0
            else
                echo "endpoint exists, get deployment that already exists "
                PROD_DEPLOYMENT=$(az ml online-endpoint show -n $DEV_ENDPT_NAME -g $(resourceGroup) -w $(wsName) -o tsv --query "tags.prod")
                echo $PROD_DEPLOYMENT
                az ml online-deployment create -g $(resourceGroup) -w $(wsName) --name $NEW_DEPLOYMENT_NAME -f "chapter 9/src/deploy/model_deployment-dev.yml" 
                #test online-deployment created with sample file.
                echo "test new endpoint"
                az ml online-endpoint invoke -n $DEV_ENDPT_NAME --deployment $NEW_DEPLOYMENT_NAME --request-file "chapter 9/data/request.json"
            
                echo "---------------------------------------------------------------------------"
                echo "update tag and traffic"
                az ml online-endpoint update -g $(resourceGroup) -w $(wsName) -n $DEV_ENDPT_NAME --set tags.prod=$NEW_DEPLOYMENT_NAME
            
                echo "---------------------------------------------------------------------------"
                echo "set traffic for origin endpoint to 0 - should be updated, but for completeness"
                az ml online-endpoint update -g $(resourceGroup) -w $(wsName) -n $DEV_ENDPT_NAME --traffic "$NEW_DEPLOYMENT_NAME=100 $PROD_DEPLOYMENT=0"  --set tags.prod=$NEW_DEPLOYMENT_NAME

                echo "---------------------------------------------------------------------------"
                echo "check online endpoints deployed"
                az ml online-endpoint list --query "[].{Name:name}"  --output table --resource-group $(resourceGroup) --workspace-name $(wsName)
                echo "delete old online endpoints deployed"
                az ml online-deployment delete -g $(resourceGroup) -w $(wsName) --endpoint $DEV_ENDPT_NAME --name $PROD_DEPLOYMENT --yes --no-wait
            fi   
        name: 'deploydevmodel'
        displayName: 'deploydevmodel'
        condition: eq(variables['checkversions.runme'], 'true')
            
      - task: CopyFiles@2
        condition: eq(variables['checkversions.runme'], 'true')
        inputs:
          sourceFolder: '$(Build.SourcesDirectory)'
          contents:  |
            '**/chapter 9/src/deploy/**' 
            '**/chapter 9/src/data/request.json' 
          targetFolder: '$(Build.ArtifactStagingDirectory)' 

      - task: PublishBuildArtifacts@1
        condition: eq(variables['checkversions.runme'], 'true')
        displayName: 'Publish Artifact: drop'
        inputs:
          ArtifactName: 'drop'
          publishLocation: 'Container'
          PathtoPublish: '$(Build.ArtifactStagingDirectory)'

- stage: 'QAPromoteModel'
  dependsOn: DevRunPipline
  condition: eq(dependencies.DevRunPipline.outputs['TrainingPipeline.checkversions.runme'], 'true')
  variables:
  - group: devops-variable-group-qa
  displayName: 'QAPromoteModel'
  jobs:
  - deployment: "RegisterModelQA"
    environment: qa
    variables:
      vardevResourceGroup: $[ stageDependencies.DevRunPipline.TrainingPipeline.outputs['setfinalversion.devResourceGroup'] ]
      vardevWsName: $[ stageDependencies.DevRunPipline.TrainingPipeline.outputs['setfinalversion.devWsName'] ]
      vardevLocation: $[ stageDependencies.DevRunPipline.TrainingPipeline.outputs['setfinalversion.devLocation'] ]
      vardevModelVersion: $[ stageDependencies.DevRunPipline.TrainingPipeline.outputs['setfinalversion.finalmodelversion'] ]
    strategy:
      runOnce:    
        deploy:
          steps:
          - download: current
            artifact: drop
          - task: PowerShell@2
            inputs:
              targetType: 'inline'
              script: |
                ls
                ls '$(Pipeline.Workspace)/drop/'

          - task: AzureCLI@1
            env:
              wsName: $(wsName)
              resourceGroup: $(resourceGroup)
              location: $(location)
            inputs:
              azureSubscription: aml-dev
              scriptLocation: inlineScript
              workingDirectory: '$(Build.SourcesDirectory)'
              inlineScript: |
                  az extension add -n ml -y
                  az version
                  echo "model version"
                  echo $(vardevModelVersion)
                  az ml model list -w $(vardevWsName) -g $(vardevResourceGroup)  -n $(model_name) --query "[0].version" -o tsv
                  az ml model download  -w $(vardevWsName) -g $(vardevResourceGroup)  -n $(model_name) -v $(vardevModelVersion)
                  ls
            name: 'downloadmodel'
            displayName: 'downloadmodel'

          - task: AzureCLI@1
            env:
              wsName: $(wsName)
              resourceGroup: $(resourceGroup)
              location: $(location)
            inputs:
              azureSubscription: aml-qa
              scriptLocation: inlineScript
              workingDirectory: '$(Build.SourcesDirectory)'
              inlineScript: |
                  echo "files:"
                  ls
                  echo "model version" $(vardevModelVersion)
                  az configure --defaults group=$(resourceGroup) workspace=$(wsName) location=$(location)
                  az ml model create --name $(model_name) -v $(vardevModelVersion) --path ./$(model_name)/$(model_name) --type mlflow_model -g $(resourceGroup) -w $(wsName)
            name: 'registermodel'
            displayName: 'registermodel'
          - task: AzureCLI@1
            env:
              wsName: $(wsName)
              resourceGroup: $(resourceGroup)
              location: $(location)
            inputs:
              azureSubscription: aml-qa
              scriptLocation: inlineScript
              workingDirectory: '$(Build.SourcesDirectory)'
              inlineScript: |
                  echo 'final model version was:'$(vardevModelVersion)
                  export NEW_DEPLOYMENT_NAME=deployment`echo $(vardevModelVersion)`v`echo $(date +"%s")`
            
                  echo $NEW_DEPLOYMENT_NAME
                  echo $ENDPT_NAME

                  az extension add -n ml -y
                  az version

                  ENDPOINT_EXISTS=$(az ml online-endpoint list -g $(resourceGroup) -w $(wsName) -o tsv --query "[?name=='$ENDPT_NAME'][name]" |  wc -l)
                   if [[ ENDPOINT_EXISTS -ne 1 ]]; then
                      echo "endpoint does not exists"
                      az ml online-endpoint create --file '$(Pipeline.Workspace)/drop/chapter 9/src/deploy/create-endpoint.yml' -g $(resourceGroup) -w $(wsName)
                      echo "creating online deployment"
                      az ml online-deployment create --name $NEW_DEPLOYMENT_NAME -f '$(Pipeline.Workspace)/drop/chapter 9/src/deploy/model_deployment.yml' -g $(resourceGroup) -w $(wsName) 
                      echo "updating online endpoint tags"
                      az ml online-endpoint update -n $ENDPT_NAME --set tags.prod=$NEW_DEPLOYMENT_NAME  --traffic "$NEW_DEPLOYMENT_NAME=100" -g $(resourceGroup) -w $(wsName)
                      exit 0
                  else
                      echo "endpoint exists, get deployment that already exists "
                      PROD_DEPLOYMENT=$(az ml online-endpoint show -n $ENDPT_NAME -g $(resourceGroup) -w $(wsName) -o tsv --query "tags.prod")
                      echo $PROD_DEPLOYMENT
                      az ml online-deployment create -g $(resourceGroup) -w $(wsName) --name $NEW_DEPLOYMENT_NAME -f "$(Pipeline.Workspace)/drop/chapter 9/src/deploy/model_deployment.yml" 
                      #test online-deployment created with sample file.
                      echo "test new endpoint"
                      az ml online-endpoint invoke -n $ENDPT_NAME --deployment $NEW_DEPLOYMENT_NAME --request-file "$(Pipeline.Workspace)/drop/chapter 9/data/request.json"
 
                      echo "---------------------------------------------------------------------------"
                      echo "update tag and traffic"
                      az ml online-endpoint update -g $(resourceGroup) -w $(wsName) -n $ENDPT_NAME --set tags.prod=$NEW_DEPLOYMENT_NAME
            
                      echo "---------------------------------------------------------------------------"
                      echo "set traffic for origin endpoint to 0 - should be updated, but for completeness"
                      az ml online-endpoint update -g $(resourceGroup) -w $(wsName) -n $ENDPT_NAME --traffic "$NEW_DEPLOYMENT_NAME=100 $PROD_DEPLOYMENT=0"  --set tags.prod=$NEW_DEPLOYMENT_NAME

                      echo "---------------------------------------------------------------------------"
                      echo "check online endpoints deployed"
                      az ml online-endpoint list --query "[].{Name:name}"  --output table --resource-group $(resourceGroup) --workspace-name $(wsName)
                      echo "delete old online endpoints deployed"
                      az ml online-deployment delete -g $(resourceGroup) -w $(wsName) --endpoint $ENDPT_NAME --name $PROD_DEPLOYMENT --yes --no-wait
                
                  fi   
            name: 'deploymodel'
            displayName: 'deploymodel'          


 

Overwriting ./src/AzureDevOpsPipeline.yml
